# 3D Model Operations

In this notebook we will focus on extracting metadata from 3D models, converting them to GLB with materials and rendering thumbnails from multiple perspectives.
<br>An example model is provided in Tutorials/3DModels, consisting of OBJ, MTL and texture files.

## Validate & Extract Metadata

Let's start with validating the OBJ model file and extracting its Metadata.
As you might know, OBJ files contain various information, which can be inspected using a simple text editor.

OBJ files, being plain text files, store a wealth of information about 3D models. Here's an overview of the key metadata that can be extracted:

- **Geometric Vertices**:
  - Prefixed with 'v' || Represent 3D coordinates (x, y, z) of each vertex in the model
- **Texture Coordinates**:
  - Prefixed with 'vt' || Define 2D coordinates (u, v) for texture mapping
- **Vertex Normals**:
  - Prefixed with 'vn' || Specify normal vectors for vertices, crucial for shading
- **Parameter Space Vertices**:
  - Prefixed with 'vp' || Used in curve or surface geometry
- **Faces**:
  - Prefixed with 'f' || Define polygons using vertex indices
- **Object Names**:
  - Prefixed with 'o' || Identify distinct objects within the file
- **Group Names**:
  - Prefixed with 'g' || Organize faces into named groups
- **Smoothing Groups**:
  - Prefixed with 's' || Control shading across faces
- **Material Libraries**:
  - Prefixed with 'mtllib' || Reference external material definition files
- **Material Names**:
  - Prefixed with 'usemtl' || Assign materials to subsequent faces
- **Comments**:
  - Prefixed with '#' || Provide additional information or annotations

By extracting this metadata, we can gain insights into the model's structure, appearance, and organization. This information is valuable for various 3D processing tasks, including validation, modification, and conversion to other formats.

In [ ]:
import os
from pathlib import Path
os.chdir(Path().absolute().parent) if Path().absolute().name == "Tutorials" else None

from file_validator import validate_file
from model_tools import convert_OBJ_to_GLB, extract_mtl_metadata, extract_obj_statistics, render_model_thumbnails
from utilities import printJSON, search_file

# Define OBJ Model Filepath
obj_filepath = Path("Tutorials/3DModels/test_model.obj")

# Extract Statistics
obj_stats = extract_obj_statistics(obj_filepath)

# Print Extracted Results
for key, value in obj_stats.items():
    print(f"{key.replace('_', ' ').title()}: {value}")

#### Extract Material Metadata
In addition to the Metadata we might add to our Zenodo Record, the `extract_obj_statistics` provides data that we can use in our validation process, e.g. to check if all linked material and texture files are available.
<br>As we can see, a MTL file is mentioned in `Material Libraries`, which we will now find in order to extract its metadata as well for further validation processes:

In [ ]:
mtl_metadata = extract_mtl_metadata(obj_filepath, obj_stats)

if mtl_metadata:
    print(f"Number of materials: {len(mtl_metadata['materials'])}")
    print(f"Referenced Textures Paths:")
    printJSON(mtl_metadata['textures'])
    
    # Print details of each material
    for material in mtl_metadata['materials']:
        print(f"\nMaterial Name: {material['name']}")
        print("Linked Textures:")
        printJSON(material['textures'])
else:
    print("Failed to extract MTL metadata.")

Let's try to find the referenced texture files and validate them as well:

In [ ]:
search_dir = Path("Tutorials/3DModels")
for texture_path in mtl_metadata['textures']:
    filename, ext = Path(texture_path).name, Path(texture_path).suffix
    found = search_file(filename, search_dir, [ext], search_subdirectories=True)
    if found:
        print(f"'{texture_path}' -> {str(found)}")
        validation_errors = validate_file(found)
        if validation_errors:
            print(f"File is invalid!: {validation_errors}")
        else:
            print("-- Validated")
    else:
        print(f"Texture not found!: '{texture_path}'")

### Convert OBJ, MTL and Textures to GLB

Now that we are sure that all referenced materials and textures are available and valid, we can convert everything into one GLB file. This has multiple advantages:

1. **Consolidation**: GLB (Binary glTF) combines geometry, materials, textures, and animations into a single binary file, simplifying asset management and distribution.

2. **Compression**: GLB files are typically smaller than the sum of their OBJ, MTL, and texture components, reducing storage requirements and improving load times.

3. **Wide Compatibility**: GLB is widely supported across various 3D platforms, game engines, and web browsers, enhancing portability and ease of use.

4. **Standardization**: As part of the glTF 2.0 specification, GLB follows a well-defined standard, ensuring consistent interpretation across different software and platforms.

5. **Efficient Rendering**: GLB files are optimized for real-time rendering, making them ideal for interactive 3D applications and WebGL-based visualizations.

6. **Metadata Support**: GLB allows for the inclusion of custom metadata, which can be useful for preserving information about the model's origin, authorship, or other relevant details.

7. **Texture Embedding**: All textures are embedded within the GLB file, eliminating the need to manage multiple separate image files.

8. **Animation Support**: If the original model includes animations, these can be preserved and efficiently encoded in the GLB format.

9. **PBR Material Support**: GLB supports Physically Based Rendering (PBR) materials, allowing for more realistic and consistent material representations across different rendering engines.

10. **Future-Proofing**: As a modern and actively maintained format, GLB is likely to have continued support and improvements, making it a good choice for long-term asset preservation.

<br>
By converting our OBJ model and its associated files to GLB, we create a more robust, portable, and efficient 3D asset that's ready for a wide range of applications and platforms.
The function `convert_OBJ_to_GLB` is able to handle searching for textures, which we have done manually in the above code cell.

We will additionally extract the GLB Metadata to see how it looks like, and if there are any divergences with the OBJ/MTL Metadata.

In [ ]:
# Convert OBJ + Materials/Textures to GLB
glb_path = convert_OBJ_to_GLB(obj_filepath, search_textures=True)
assert glb_path, f"Failed to convert to GLB"

# Extract GLB Metadata
from model_tools import extract_glb_metadata
glb_metadata = extract_glb_metadata(glb_path)
print("GLB Metadata:")
printJSON(glb_metadata)

### Render Model Thumbnails

By utilizing the [Blender Python API](https://docs.blender.org/api/current/index.html), we are able to render Thumbnails with lots of possibilities regarding settings, artificial lights etc.
<br>The function `render_model_thumbnails` is designed to always center the object/scene, and to alter the point of camera view in a logical way.

Let's try to render three thumbnails for our test model:

In [ ]:
render_model_thumbnails(glb_path, num_perspectives=3, include_top_perspective=False, 
                        resolution=1000, use_gpu=False, resize=False)

### Key Parameters

Now, it is your turn to explore the function and modify the parameters, here are some explanations:

**Perspective Control**:
- `num_perspectives`: Number of side views to render (default: 4)
- `include_top_perspective`: Whether to include a top-down view (default: True)

**Camera Settings**:
- `camera_distance_factor`: Multiplier for camera distance from the object (default: 2.0)
  - Higher values place the camera further from the object

**Lighting**:
- `light_type`: Type of light source (default: "SUN")
- `light_energy`: Intensity of the light (default: 5.0)
- `light_angle`: Angle of the light in degrees (default: 112.0)
  - For "SUN" type, this affects the softness of shadows

**Render Quality**:
- `resolution`: Size of the rendered image in pixels (default: 1000)
- `samples`: Number of samples for rendering (default: 128)
  - Higher values increase quality but also render time

**GPU Rendering**:
- `use_gpu`: Whether to use GPU for rendering (default: False)
- `gpu_device_type`: GPU computation type (default: "CUDA")

**Post-processing**:
- `resize`: Whether to create smaller versions of the render (default: False)
- `resize_dimensions`: List of sizes for resized images (default: [512, 256, 128])

#### Perspective Calculation
- **Side Views**: Evenly spaced around object (e.g., 0°, 90°, 180°, 270° for 4 views)
- **Top View**: Directly above object center (optional)
- **Camera Angle**: 60° downward for side views, 90° for top view
- **Positioning**: Based on object's bounding box and `camera_distance_factor`
- **Light**: Placed at camera position for consistent illumination
